# Jordan-Wigner Transformation for LiH

This script constructs the molecular Hamiltonian of lithium hydride (LiH) using PySCF, performs a Hartree-Fock and CASCI calculation to extract the one- and two-electron integrals, and then maps the resulting fermionic Hamiltonian to a qubit (spin) Hamiltonian via the Jordan-Wigner transformation. The transformation is implemented explicitly using Qiskit's `SparsePauliOp` representation of Pauli strings, allowing for efficient quantum simulation on qubit-based hardware. The result is a sparse spin Hamiltonian suitable for variational algorithms or quantum eigensolvers, and the script outputs both the operator and the number of nonzero Pauli terms involved.

In [ ]:
import itertools

import numpy as np
import pyscf
from pyscf import ao2mo, gto, mcscf
from qiskit.quantum_info import SparsePauliOp

# Build LiH molecule using PySCF
mol = gto.Mole()
mol.build(
    verbose=0,
    atom=[["Li", (0, 0, 0)], ["H", (0, 0, 1.59)]],
    basis="sto-6g",
    spin=0,
    charge=0,
    symmetry="Coov",
)

# Run Hartree-Fock
scf = pyscf.scf.RHF(mol)
hf_energy = scf.kernel()

# Define active space and get molecular integrals
casci = mcscf.CASCI(scf, ncas=3, nelecas=(1, 1))
cas_space_symmetry = {"A1": 3}
mo = mcscf.sort_mo_by_irrep(casci, scf.mo_coeff, cas_space_symmetry)
casci.kernel(mo)
h1e, ecore = casci.get_h1eff()
h2e = ao2mo.restore(1, casci.get_h2eff(), casci.ncas)

# If you want to visualize the orbitals, run these lines
# import pyscf.tools
# for i in range(1, 1 + casci.ncas):
#     pyscf.tools.cubegen.orbital(
#         mol, "casscf_%d_%.1f.cube" % (i, scf.mo_occ[i]), casci.mo_coeff[:, i]
#     )


def _qubit_create(qubit: int, num_qubits: int):
    """Creation operator under Jordan-Wigner transformation."""
    qubits = list(range(qubit + 1))
    return SparsePauliOp.from_sparse_list(
        [("Z" * qubit + "X", qubits, 0.5), ("Z" * qubit + "Y", qubits, -0.5j)],
        num_qubits=num_qubits,
    )


def _qubit_destroy(qubit: int, num_qubits: int):
    """Destruction operator under Jordan-Wigner transformation."""
    qubits = list(range(qubit + 1))
    return SparsePauliOp.from_sparse_list(
        [("Z" * qubit + "X", qubits, 0.5), ("Z" * qubit + "Y", qubits, 0.5j)],
        num_qubits=num_qubits,
    )


def jordan_wigner(
    h1e: np.ndarray, h2e: np.ndarray, ecore: float, tol: float = 1e-8
) -> SparsePauliOp:
    """Apply Jordan-Wigner transformation to a molecular Hamiltonian."""
    norb, _ = h1e.shape
    create_ops = [_qubit_create(i, 2 * norb) for i in range(2 * norb)]
    destroy_ops = [_qubit_destroy(i, 2 * norb) for i in range(2 * norb)]
    op = SparsePauliOp.from_sparse_list([("", [], ecore)], num_qubits=2 * norb)
    for p, r in itertools.product(range(norb), repeat=2):
        coeff = h1e[p, r]
        for sigma in range(2):
            op += coeff * create_ops[norb * sigma + p] @ destroy_ops[norb * sigma + r]
    for p, r, q, s in itertools.product(range(norb), repeat=4):
        coeff = 0.5 * h2e[p, r, q, s]
        for sigma, tau in itertools.product(range(2), repeat=2):
            op += (
                coeff
                * create_ops[norb * sigma + p]
                @ create_ops[norb * tau + q]
                @ destroy_ops[norb * tau + s]
                @ destroy_ops[norb * sigma + r]
            )
    return op.simplify(atol=tol)


hamiltonian = jordan_wigner(h1e, h2e, ecore)

print(hamiltonian)
print("Number of Pauli terms:", len(hamiltonian))